In [ ]:
pip install --upgrade openai

In [37]:
import pandas as pd

# read xlsx file
df = pd.read_excel("/Users/johri/Projects/AIEarthHack/AI EarthHack Dataset.xlsx")
print(df.columns)
print("Before pre-processing: ", len(df))

# filter entries with less than 10 words in the solution column
df = df[df['solution'].str.split().str.len().gt(20)]

# filter entries with similar solution 
df = df.drop_duplicates(subset=['solution'])
print("After pre-processing: ", len(df))

Index(['id', 'problem', 'solution'], dtype='object')
Before pre-processing:  1300
After pre-processing:  1208


In [76]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="API_KEY",
)

def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        response_format={"type":"json_object"},
        messages=[
            {"role": "system", "content": "Provide output in valid JSON"},
            {"role": "user", "content": prompt}]
    )

    ret_json = response.choices[0].message.content.strip()
    
    return ret_json


## if using different version of openai
# import openai
# def chat_gpt(prompt):
#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": prompt}]
#     )
#     ret_json = json.dumps(response.choices[0].message.content.strip())
    
#     return ret_json


In [91]:
from ast import literal_eval

min_word_count = 10
min_bullet = 2
max_bullet = 5
key_features = "objective, relevance_to_circular_economy, feasibility, impact, technical_complexity, innovation, sustainability, market_potential, competitive_landscape, cost-effectiveness"
dict_keys = "'rating_objective', 'rating_relevance_to_circular_economy', 'rating_feasibility', 'rating_impact', 'rating_technical_complexity', 'rating_innovation', 'rating_sustainability', 'rating_market_potential', 'rating_competitive_landscape', 'rating_cost-effectiveness', 'explanation_objective', 'explanation_relevance_to_circular_economy', 'explanation_feasibility', 'explanation_impact', 'explanation_technical_complexity', 'explanation_innovation', 'explanation_sustainability', 'explanation_market_potential', 'explanation_competitive_landscape', 'explanation_cost-effectiveness'"
df_final = pd.DataFrame()
n = 5

for inx in range(0, 10, n):
    proposals = ""
    selected_df = df[inx:inx+n]
    for inx, row in selected_df.iterrows():
        problem_statement = row['problem'].strip()
        solution_statement = row['solution'].strip()
        proposals += f"id: {row['id']}\nproblem: {problem_statement}\nsolution: {solution_statement}\n"

    prompt = f"""Given are {n} proposals for a competition titled "Unlocking the Potential of Circular Economy".
                The solutions were sourced all over the world in various ways with circular economy applications to a wide array of industries, ranging from textiles to food waste management.
                Participants were asked about the problem their solution is meant to solve and describe the solution in their own words.
                These solutions must be presented before a human judge. To help the judge, extract the main idea from each proposal.
                Secondly, for each proposal provide comments and ratings using the following rubrics: {key_features}
                The comments for any proposal must not exceed {min_word_count} words. Limit to {min_bullet}-{max_bullet} bullet points. Rate each rubric from 0 to 100, with 0 being the worst and 100 being the best.
                Thirdly, generate a summary of the description of problem and solution for each propsal. Limit the summary to around 100 words.
                The output should be a single json as mentioned below: 
                {{
                    id1: {{
                        explanation_rubric1: ...,
                        rating_rubric1: ...,
                        explanation_rubric2: ...,
                        rating_rubric2: ...,
                        ...
                        summary: ...
                    }},
                    id2: {{
                        explanation_rubric1: ...,
                        rating_rubric1: ...,
                        explanation_rubric2: ...,
                        rating_rubric2: ...,
                        ...
                        summary: ... 
                        }},
                    ...
                }}

                Here, id1, id2, ... will be the respective proposal id.
                The rubric1, rubric2, ... will be: {dict_keys}.
                Ensure that the summary describes  the problem and solution clearly. 
                
                Use the following pairs of problem and solution enclosed in triple backticks:
                ```
                {proposals}
                ```
                """

    # print(prompt)
    response_json = chat_gpt(prompt)

    print(response_json)
    # json_object = literal_eval(json.loads(response_json))
    json_object = json.loads(response_json)

    json_df = pd.DataFrame(json_object).T
    df_final = pd.concat([df_final, json_df], ignore_index=True)

df_final

{
    "1": {
        "rating_objective": 90,
        "explanation_objective": "Addresses high construction waste and resource depletion",
        "rating_relevance_to_circular_economy": 95,
        "explanation_relevance_to_circular_economy": "Modular construction promotes reuse and recycling",
        "rating_feasibility": 85,
        "explanation_feasibility": "Proven feasibility in global markets",
        "rating_impact": 92,
        "explanation_impact": "Significantly reduces construction waste and time",
        "rating_technical_complexity": 80,
        "explanation_technical_complexity": "Moderate technical complexity",
        "rating_innovation": 88,
        "explanation_innovation": "Innovative approach to traditional construction methods",
        "rating_sustainability": 94,
        "explanation_sustainability": "Drives the industry towards a sustainable future",
        "rating_market_potential": 85,
        "explanation_market_potential": "Scalability and real-world app

,rating_objective,explanation_objective,rating_relevance_to_circular_economy,explanation_relevance_to_circular_economy,rating_feasibility,explanation_feasibility,rating_impact,explanation_impact,rating_technical_complexity,explanation_technical_complexity,...,explanation_innovation,rating_sustainability,explanation_sustainability,rating_market_potential,explanation_market_potential,rating_competitive_landscape,explanation_competitive_landscape,rating_cost-effectiveness,explanation_cost-effectiveness,summary
0,90,Addresses high construction waste and resource...,95,Modular construction promotes reuse and recycling,85,Proven feasibility in global markets,92,Significantly reduces construction waste and time,80,Moderate technical complexity,...,Innovative approach to traditional constructio...,94,Drives the industry towards a sustainable future,85,Scalability and real-world application,86,Competitive in the construction industry,91,Substantial financial savings for businesses,Modular construction reduces waste by 90% and ...
1,91,Addresses the need for efficient renewable energy,80,Promotes green revolution and efficient energy...,88,"Utilizes existing, under-utilized technology",85,"Creates a future of clean, quiet, and efficien...",82,Moderate technical complexity,...,Revolutionary approach to harness wind energy,87,Driving a green revolution from the ground up,84,Potential for commercial scale wind energy gen...,83,Can drive a green revolution,86,Efficient use of existing technology,Revolutionary approach to harness wind energy ...
2,88,Addresses carbon footprint and waste from outd...,90,Promotes reuse and sharing resources,85,Integrates gamification and efficient resource...,89,Contributes to cultural shift and waste reduction,80,Moderate technical complexity,...,Innovative book exchange platform,92,Contributes to circular economy and waste redu...,85,Potential for broader cultural and societal shift,83,Leverages gamification and efficient resource ...,88,Saves money for students and schools,A Book Swap program promoting the reuse of boo...
3,90,Addresses unsustainable clothing production an...,92,Transforms fashion industry into a subscriptio...,85,"Dependent on location, culture, and income level",88,"Reduces garment production, transportation, an...",80,Moderate technical complexity,...,Promotes a continuous income stream and sustai...,92,Maximizes the life cycle of garments and reduc...,87,Creates a continuous income stream for businesses,86,Transforms the fashion industry model,90,Creates a sustainable and continuous revenue s...,A garment rental service transforming the fash...
4,94,Addresses electronic waste and resource underu...,93,Promotes modular model for electronic devices,88,Dependent on continuous technological advancem...,92,Promotes resource use efficiency and waste red...,85,Moderate technical complexity with continuous ...,...,Maximizes the life and value of electronic gad...,95,Promotes resource efficiency and economic viab...,86,Creates a continuous market space for recycled...,88,Increases customer loyalty and engagement,89,Continuous customer engagement and value capture,A modular electronic device model promoting re...
5,85,Solves paper waste and operational costs,90,"Reduces paper consumption, promotes resource c...",80,"High scalability, feasible with digital tech",88,"Reduces paper waste, environmental and financi...",75,Involves the development of a secure mobile app,...,"Introduces the 'tap-and-go' feature, enhancing...",85,"Significantly reduces paper consumption, promo...",90,"Appeals to businesses of all sizes, high scala...",82,Offers substantial savings on paper and printi...,85,"Substantial savings on paper, printing, and st...",The proposal aims to reduce paper waste and op...
6,88,Solves parking and carsharing challenges,83,"Reduces driving distances, fuel consumption",85,Feasible with three-part strategy,90,"Improves carsharing experience, reduces costs,...",82,Involves the implementation

In [95]:
result_df = pd.concat([df[['id']], df_final], axis=1) 
# result_df.to_csv("result.csv")
result_df

,id,rating_objective,explanation_objective,rating_relevance_to_circular_economy,explanation_relevance_to_circular_economy,rating_feasibility,explanation_feasibility,rating_impact,explanation_impact,rating_technical_complexity,...,explanation_innovation,rating_sustainability,explanation_sustainability,rating_market_potential,explanation_market_potential,rating_competitive_landscape,explanation_competitive_landscape,rating_cost-effectiveness,explanation_cost-effectiveness,summary
0,1.0,90,Addresses high construction waste and resource...,95,Modular construction promotes reuse and recycling,85,Proven feasibility in global markets,92,Significantly reduces construction waste and time,80,...,Innovative approach to traditional constructio...,94,Drives the industry towards a sustainable future,85,Scalability and real-world application,86,Competitive in the construction industry,91,Substantial financial savings for businesses,Modular construction reduces waste by 90% and ...
1,2.0,91,Addresses the need for efficient renewable energy,80,Promotes green revolution and efficient energy...,88,"Utilizes existing, under-utilized technology",85,"Creates a future of clean, quiet, and efficien...",82,...,Revolutionary approach to harness wind energy,87,Driving a green revolution from the ground up,84,Potential for commercial scale wind energy gen...,83,Can drive a green revolution,86,Efficient use of existing technology,Revolutionary approach to harness wind energy ...
2,3.0,88,Addresses carbon footprint and waste from outd...,90,Promotes reuse and sharing resources,85,Integrates gamification and efficient resource...,89,Contributes to cultural shift and waste reduction,80,...,Innovative book exchange platform,92,Contributes to circular economy and waste redu...,85,Potential for broader cultural and societal shift,83,Leverages gamification and efficient resource ...,88,Saves money for students and schools,A Book Swap program promoting the reuse of boo...
3,4.0,90,Addresses unsustainable clothing production an...,92,Transforms fashion industry into a subscriptio...,85,"Dependent on location, culture, and income level",88,"Reduces garment production, transportation, an...",80,...,Promotes a continuous income stream and sustai...,92,Maximizes the life cycle of garments and reduc...,87,Creates a continuous income stream for businesses,86,Transforms the fashion industry model,90,Creates a sustainable and continuous revenue s...,A garment rental service transforming the fash...
4,5.0,94,Addresses electronic waste and resource underu...,93,Promotes modular model for electronic devices,88,Dependent on continuous technological advancem...,92,Promotes resource use efficiency and waste red...,85,...,Maximizes the life and value of electronic gad...,95,Promotes resource efficiency and economic viab...,86,Creates a continuous market space for recycled...,88,Increases customer loyalty and engagement,89,Continuous customer engagement and value capture,A modular electronic device model promoting re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,1296.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1296,1297.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,1299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,1300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
